In [12]:
import requests
from bs4 import BeautifulSoup as bs
import json
import time
import os

In [13]:
nowdate=time.strftime("%Y%m%d", time.localtime())

In [14]:
b=time.time()
URL="http://www.104.com.tw/i/apis/jobsearch.cfm?fmt=8&cols=JOB,JOB_ADDR_NO_DESCRIPT,NAME,JOB_ADDRESS,JOBCAT_DESCRIPT,APPEAR_DATE,SAL_MONTH_LOW,SAL_MONTH_HIGH,LAT,LON&pgsz=2000&order=5&cat=2006001000&asc=1"
#待加入area=60010XX0XX

#市代號
bigAreaList=[len(str(i))<2 and "0"+str(i) or str(i) for i in range(1,14+1)] 
#區代號
samllAreaList=[len(str(i))<2 and "0"+str(i) or str(i) for i in range(1,29+1)] 

savedir="../data/104json"

if not os.path.exists(savedir):
    os.makedirs(savedir)

if not os.path.exists(savedir+"/"+nowdate):
    os.makedirs(savedir+"/"+nowdate)
    

finishArea=[]
# finishArea=[file.split(".")[0] for file in os.listdir(savedir)] 


some={'中餐廚師',
 '儲備幹部',
 '其他類廚師',
 '工讀生',
 '店長／賣場管理人員',
 '日式廚師',
 '櫃檯接待人員',
 '洗碗人員',
 '營養師',
 '物管／資材',
 '生鮮人員',
 '經營管理主管',
 '行政助理',
 '西餐廚師',
 '西點／蛋糕師',
 '記帳／出納／一般會計',
 '調酒師／吧台人員',
 '連鎖店管理人員',
 '門市／店員／專櫃人員',
 '領班',
 '食品研發人員',
 '食品衛生管理師',
 '飯店工作人員',
 '飯店或餐廳主管',
 '餐廚助手',
 '餐飲服務生',
 '麵包師'}


def get104JsonData(allpage,url):
    '''爬取這個網頁的json檔'''
    
    HRdata104=[]
    for i in range(1,allpage+1):
        res=requests.get(url+"&page=%s"%i)
        if res.status_code==200:
            alldata=json.loads(res.text)
            HRdata104+=[case for case in alldata['data'] if case["NAME"]!="104外包網" 
                        and case['JOBCAT_DESCRIPT'].split("@")[0] in some]
        res.close()
        time.sleep(1)
        print(i)
    for dien in HRdata104:
        dien['JOBCAT_DESCRIPT']=dien['JOBCAT_DESCRIPT'].split("@")[0]
    return HRdata104

#找大區域資料
for bigkey in bigAreaList:
    area="60010%s000"%bigkey
    
    #判斷是否已有資料
    if area not in finishArea:
        URL2=URL+"&area=%s"%area
        res=requests.get(URL2)
        time.sleep(1)
        
        #判斷是否連上
        if res.status_code==200:
            data=json.loads(res.text)
            allpage=int(data["TOTALPAGE"])
            
            #判斷是否有該頁資料
            if allpage!=0:
                
                #判斷資料是否小於三千(104一次只能顯示20頁(3000筆) 多的不給抓)
                if allpage<21:
                    HRdata104=get104JsonData(allpage,URL2)
                    with open(savedir+"/"+nowdate+"/"+area+".json","w") as f:
                        json.dump(HRdata104,f)
                    print(area)
                #如果資料大於三千，則抓小區域資料
                else:
                    for smallkey in samllAreaList:
                        area="60010%s0%s"%(bigkey,smallkey)
                        if area not in finishArea:
                            URL2=URL+"&area=%s"%area
                            res=requests.get(URL2)
                            time.sleep(1)
                            if res.status_code==200:
                                data=json.loads(res.text)
                                allpage=int(data["TOTALPAGE"])
                                if allpage!=0:
                                    HRdata104=get104JsonData(allpage,URL2)
                                    with open(savedir+"/"+nowdate+"/"+area+".json","w") as f:
                                        json.dump(HRdata104,f)
                                    print(area)
e=time.time()
print(e-b)

1
2
3
4
5
6
6001001001
1
2
6001001002
1
2
3
4
5
6
7
8
9
6001001003
1
2
3
4
5
6
6001001004
1
2
3
4
5
6
7
8
9
10
11
6001001005
1
2
6001001006
1
2
3
4
5
6
7
8
9
6001001007
1
2
3
4
6001001008
1
2
6001001009
1
2
3
4
5
6001001010
1
2
3
6001001011
1
2
6001001012
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
6001002000
1
2
6001003000
1
6001004000
1
2
3
4
5
6
7
8
9
10
11
12
13
6001005000
1
2
3
4
5
6
7
8
9
6001006000
1
2
6001007000
1
6001008001
1
6001008002
1
6001008003
1
2
3
4
6001008004
1
2
3
6001008005
1
2
6001008006
1
2
3
4
5
6
6001008007
1
2
3
6001008008
1
6001008009
1
6001008010
1
6001008011
1
6001008012
1
6001008013
1
6001008014
1
6001008016
1
6001008017
1
6001008018
1
6001008019
1
6001008020
1
6001008021
1
6001008022
1
6001008023
1
6001008024
1
6001008025
1
6001008026
1
6001008027
1
6001008028
1
2
6001010000
1
6001011000
1
2
6001012000
1
2
6001013000
1
2
3
4
5
6
7
8
6001014000
368.6790871620178


In [19]:
HRdata104[10]

{'APPEAR_DATE': '20171201',
 'JOB': '勝博殿內/外場服務員[台南新光三越中山店]',
 'JOBCAT_DESCRIPT': '餐飲服務生',
 'JOB_ADDRESS': '中山路162號[新光三越中山店內]',
 'JOB_ADDR_NO_DESCRIPT': '台南市中西區',
 'LAT': '22.995709',
 'LON': '120.20994',
 'NAME': '勝博殿_勝成餐飲股份有限公司',
 'SAL_MONTH_HIGH': '024000',
 'SAL_MONTH_LOW': '021280'}

In [16]:
import os
import json
savedir="../data/104json"+"/"+nowdate
finishArea=[file.split(".")[0] for file in os.listdir(savedir)]
datalist=[json.load(open(savedir+"/"+file)) for file in os.listdir(savedir)]


def flat_list_v_1(the_list):
    is_nested = True
    before = the_list[:]
    while is_nested:
        is_nested = False
        now = []
        for item in before:
            if isinstance(item, list):
                now.extend(item)
                is_nested = True
            else:
                now.append(item)
        before = now
    return before

alldata=flat_list_v_1(datalist)

In [17]:
len(alldata)

25404

In [18]:
import pymongo
from pymongo import MongoClient
import datetime
import json
import time
from functools import reduce

#連Mongodb
client = MongoClient('172.20.26.39', 27017)
#連DB名
db=client['rawData']
#秀DB內的collection list
db.collection_names()
#選擇我們想要的collection
collection = db.HRdata104
collection.drop()
collection.insert_many(alldata)

In [22]:
some={'中餐廚師',
 '儲備幹部',
 '其他類廚師',
 '工讀生',
 '店長／賣場管理人員',
 '日式廚師',
 '櫃檯接待人員',
 '洗碗人員',
 '營養師',
 '物管／資材',
 '生鮮人員',
 '經營管理主管',
 '行政助理',
 '西餐廚師',
 '西點／蛋糕師',
 '記帳／出納／一般會計',
 '調酒師／吧台人員',
 '連鎖店管理人員',
 '門市／店員／專櫃人員',
 '領班',
 '食品研發人員',
 '食品衛生管理師',
 '飯店工作人員',
 '飯店或餐廳主管',
 '餐廚助手',
 '餐飲服務生',
 '麵包師'}

In [20]:
HRdata104

[{'APPEAR_DATE': '20171201',
  'JOB': '◇計時夥伴-台南市門市◇',
  'JOBCAT_DESCRIPT': '餐飲服務生',
  'JOB_ADDRESS': '',
  'JOB_ADDR_NO_DESCRIPT': '台南市',
  'LAT': '22.9997281',
  'LON': '120.2270277',
  'NAME': 'COMEBUY_長沂國際實業股份有限公司',
  'SAL_MONTH_HIGH': '023200',
  'SAL_MONTH_LOW': '021280'},
 {'APPEAR_DATE': '20171130',
  'JOB': '天吉屋【台南三越西門店】外場正兼職人員',
  'JOBCAT_DESCRIPT': '餐飲服務生',
  'JOB_ADDRESS': '台南市中區西門路1段658號6樓(台南新光三越西門店)',
  'JOB_ADDR_NO_DESCRIPT': '台南市',
  'LAT': '22.986809',
  'LON': '120.197803',
  'NAME': 'MoMo Paradise / Bellini / 天吉屋_三澧企業股份有限公司',
  'SAL_MONTH_HIGH': '000000',
  'SAL_MONTH_LOW': '000000'},
 {'APPEAR_DATE': '20171201',
  'JOB': '◇計時夥伴-台南善化門市◇',
  'JOBCAT_DESCRIPT': '餐飲服務生',
  'JOB_ADDRESS': '善化區中山路315-1號',
  'JOB_ADDR_NO_DESCRIPT': '台南市',
  'LAT': '23.1319421',
  'LON': '120.2951753',
  'NAME': 'COMEBUY_長沂國際實業股份有限公司',
  'SAL_MONTH_HIGH': '023200',
  'SAL_MONTH_LOW': '021280'},
 {'APPEAR_DATE': '20171201',
  'JOB': '☆正職人員-台南善化門市☆',
  'JOBCAT_DESCRIPT': '餐飲服務生',
  'JOB_ADDRES

In [22]:
HRdata104=list(collection.find({}))
print("讀取完畢")
[i.pop("_id") for i in HRdata104]
for dien in HRdata104:
    dien['JOBCAT_DESCRIPT']=dien['JOBCAT_DESCRIPT'].split("@")[0]
# HRdata104=[dien for dien in HRdata104 if dien['JOBCAT_DESCRIPT'] in some and
#            dien['JOB_ADDRESS']!="" and dien['SAL_MONTH_HIGH']!='000000' and dien["LAT"]!="0"]  
# HRdata104=[dien for dien in HRdata104 if dien['JOBCAT_DESCRIPT'] in some]  
    
import pandas as pd
df = pd.DataFrame(HRdata104)
writer = pd.ExcelWriter('../data/104_'+nowdate+'.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()
print("儲存完畢")

讀取完畢
儲存完畢


In [17]:
# for dien in HRdata104:
#     dien['JOBCAT_DESCRIPT']=dien['JOBCAT_DESCRIPT'].split("@")[0]

In [20]:
# import pandas as pd
# df = pd.DataFrame(HRdata104)
# writer = pd.ExcelWriter('../data/104_'+nowdate+'.xlsx', engine='xlsxwriter')
# df.to_excel(writer, sheet_name='Sheet1')
# writer.save()
# print("儲存完畢")

儲存完畢


# 中文數字轉阿拉伯的方法

In [73]:
dict ={u'零':0, u'一':1, u'二':2, u'三':3, u'四':4, u'五':5, u'六':6, u'七':7, u'八':8, u'九':9, u'十':10, u'百':100, u'千':1000, u'万':10000,
       u'０':0, u'１':1, u'２':2, u'３':3, u'４':4, u'５':5, u'６':6, u'７':7, u'８':8, u'９':9,
       u'0':0, u'1':1, u'2':2, u'3':3, u'4':4, u'5':5, u'6':6, u'7':7, u'8':8, u'9':9,
       u'壹':1, u'贰':2, u'叁':3, u'肆':4, u'伍':5, u'陆':6, u'柒':7, u'捌':8, u'玖':9, u'拾':10, u'什':10, u'佰':100, u'仟':1000, u'萬':10000,
       u'兩':2,u'貳':2, u'參':3, u'肆':4, u'伍':5, u'陸':6, u'佰':100, u'仟':1000, u'萬':10000,
       u'亿':100000000,u'億':100000000,u'兆':1000000000000}
def getResultForDigit(a, encoding="utf-8"):
    if not isinstance(a, str):
        a = a.decode(encoding)
    count = 0
    result = 0
    tmp = 0
    Billion = 0
    while count < len(a):
        tmpChr = a[count]
        #print tmpChr
        tmpNum = dict.get(tmpChr, None)
        #如果等于1亿
        if tmpNum is None:
            pass
        elif tmpNum == 100000000:
            result = result + tmp
            result = result * tmpNum
            #获得亿以上的数量，将其保存在中间变量Billion中并清空result
            Billion = Billion * 100000000 + result
            result = 0
            tmp = 0
        #如果等于1万
        elif tmpNum == 10000:
            result = result + tmp
            result = result * tmpNum
            tmp = 0
        #如果等于十或者百，千
        elif tmpNum >= 10:
            if tmp == 0:
                tmp = 1
            result = result + tmpNum * tmp
            tmp = 0
        #如果是个位数
        elif tmpNum is not None:
            tmp = tmp * 10 + tmpNum
        count += 1
    result = result + tmp
    result = result + Billion
    return result

In [95]:
import decimal
a = decimal.Decimal('1.0')
a 

Decimal('1.0')

In [103]:
#用[a]如果沒有key a會產生error 但用dict.get(a,b) 如果key a 不存在 則會回傳b值
dict["j3"]
dict.get("j3","sss")

'sss'

In [109]:
(111 if 3>4 else 222)

222

In [126]:
lts = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [127]:
[ele for lt in lts for ele in lt]

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [139]:
#檔案有tryyyy.py 裡面有def函數跟某些數值
import tryyyy


In [142]:
print(tryyyy.max(10,30),tryyyy.pi)


30 3.141592653589793


In [145]:
from tryyyy import pi

pi

3.141592653589793

In [151]:
import bank.bank
acct = bank.account('Justin', '123-4567', 1000)
bank.deposit(acct, 500)
bank.withdraw(acct, 200)
print (bank.to_str(acct))

Account:{'name': 'Justin', 'number': '123-4567', 'balance': 1300}


In [157]:
import bank.bank2

acct = bank2.Account('Justin', '123-4567', 1000)
acct.deposit(500)
acct.withdraw(200)
print (acct)

Account(Justin, 123-4567, 1300)


In [10]:
listt = ['Justin', 'caterpillar', 'openhome']

In [18]:
for i,j in enumerate(listt):
    print(i,j)

0 Justin
1 caterpillar
2 openhome


In [17]:
for i,j in zip(range(0,3),listt):
    print(i,j)

0 Justin
1 caterpillar
2 openhome


In [21]:
from functools import reduce
reduce(lambda sum, elem: sum + elem, [1, 2, 3, 4, 5], 0)





15

In [26]:
#lambda用法   lambda  a, b, c: you want to do from a,b,c
sum=lambda o, elem: o + elem

In [30]:
#reduce用法 (公式 ,需相同計算方式的list,計算初始)
reduce(sum,[1,2,3,4,5,6],5)
#5+1+2+3+4+5+6

26

In [31]:
p=lambda o, elem: o *elem
reduce(p,[3,3,3,3],1)

#>1*3*3*3*3

81

In [150]:
import tkinter as tk
 
win=tk.Tk()
win.title("My First Tk GUI")#給標題
label=tk.Label(win, text="Hello World!")
label.pack() #顯上label
button=tk.Button(win, text="OK")
button.pack()#顯示botton
win.resizable(0,0)#禁止調整大小
win.mainloop()

In [152]:
from tkinter import *
win=Tk()
win.title("Tk GUI")
label=Label(win, text="Hello World!")
button=Button(win, text="OK")
label.pack()
button.pack()
win.mainloop()

In [158]:
from tkinter import *
win=Tk()
win.title("Tk GUI")
label=Label(win, text="Hello World!")
button=Button(win, text="OK")
label.grid(column=0,row=0)
button.grid(column=0,row=2)
win.mainloop() 

In [159]:
from tkinter import *
win=Tk()
win.title("Tk GUI")
label=Label(win, text="Hello World!")
count=0
def clickOK():
    global count
    count=count + 1
    label.configure(text="Click OK " + str(count) + " times")
button=Button(win, text="OK", command=clickOK)
label.pack()
button.pack()
win.mainloop()

In [147]:
# 1. 實作⼀一個在英⽂文句句⼦子中， 反轉單字字元順序的功能 ⼀一段英⽂文句句⼦子
# 反轉字元順序後的句句⼦子

# ex. [I]:   My name is Michele
#      [O]: yM eman si elehciM

Q="My name is Michele"

print(" ".join([i[::-1] for i in Q.split(" ")]))


# 2.回家函式練習題：
# 製作⼀一個叫做concatDict的函式， 這個函式可以傳入⼆二個字典a, b， 函式會回傳⼀一個串串接後的新字典 


# def generateDict(a,b):
# resultDict = {}
# for val in range(a, b):
# resultDict[val]=val**2
# return resultDict

# [I]:concatDict(generateDict(1,3), generateDict(6,10)) [O] : {1: 1, 2: 4, 6: 36, 7: 49, 8: 64, 9: 81}

def generateDict(a,b):
    resultDict = {}
    for val in range(a, b):
        resultDict[val]=val**2
    return resultDict

def concatDict(dict1,dict2):
    dict1.update(dict2)
    return dict1



# 3. 製作⼀一個叫做concatAllDict的函式， 這個函式可以傳入不定數量量的字典， 函式會回傳⼀一個串串接後的新字典
# (沿⽤用generateDict函式產⽣生測試資料)

# [I]:concatDict(generateDict(1,3), generateDict(6,10)) [O] : {1: 1, 2: 4, 6: 36, 7: 49, 8: 64, 9: 81}

# def concatAllDict(???):     ???

def concatAllDict(dict1,*dicts):
    for dictN in dicts:
        dict1.update(dictN)
    return dict1



yM eman si elehciM


In [148]:
concatDict(generateDict(1,3), generateDict(6,10))

{1: 1, 2: 4, 6: 36, 7: 49, 8: 64, 9: 81}

In [149]:
x=generateDict(1,3)

In [144]:
print(generateDict(1,3).update(generateDict(4,8)))

None


In [142]:
x

{1: 1, 2: 4, 4: 16, 5: 25, 6: 36, 7: 49}